# 🧹 **Limpieza del archivo**   ✨
_____

- Librerías y extender visualización

In [57]:
import pandas as pd
import numpy as np
import re 
from datetime import datetime

# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score

# ------------------------------------------------------------------------------
import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest
import warnings
warnings.filterwarnings("ignore")

# ------------------------------------------------------------------------------
pd.set_option('display.max_rows', 100)  # Cambia el número máximo de filas mostradas
pd.set_option('display.max_columns', None)  # Cambia el número máximo de columnas mostradas
pd.set_option('display.width', 1000)  # Cambia el ancho máximo de la pantalla

- **Lectura** de archivos **CSV**

In [58]:
df_hr = pd.read_csv("HR RAW DATA.csv",index_col=0)

## <span style="color:#00bf63">**Funciones**</span>
___
### 1. Función de cambios en <span style="color:#00bf63">**columnas**</span>

#### 1.1 Cambios de <u>**nombres**</u>

In [59]:
# Renombrar columnas a formato 'snake_case' manualmente
renombrar_columnas = {
    'NUMCOMPANIESWORKED': 'num_companies_worked', # mayúscula a minúscula
    'TOTALWORKINGYEARS': 'total_working_years', # mayúscula a minúscula
    'WORKLIFEBALANCE': 'work_life_balance', # mayúscula a minúscula
    'YEARSWITHCURRMANAGER': 'years_with_curr_manager', # mayúscula a minúscula
    'NUMBERCHILDREN': 'number_children', # mayúscula a minúscula
    'DateBirth': 'year_birth', # cambio a year porque los valores no indican una fecha complete
    'employeenumber': 'employee_number', # introducción de guión bajo como separación
    'employeecount': 'employee_count', # introducción de guión bajo como separación
    'MonthlyIncome' : 'anual_income', # Cliente se ha equivocado en la recogida y MonthlyIncome, corresponde a salario neto anual
    'Salary': 'monthly_income' # Se reutiliza la columna para meter el bruto mensual asi como indicado por el cliente
}

# Aplicar los cambios
df_hr.rename(columns=renombrar_columnas, inplace=True)


# Modificar columnas que tienen formato 'CamelCase' a 'snake_case'
df_hr.columns = [re.sub(r'(?<!^)(?=[A-Z])', '_', col).lower() for col in df_hr.columns]

#### 1.2. Cambios <u>eliminanando columnas</u>

In [60]:
# Justificación:
    #"employee_count" --> informacion irrelevante para hacer recuento ya que contamos con la columna "employee_number" que ademas de asignar un id tambien hace la funcion de recuento
    #"same_as_monthly_income" --> es la misma columna 
    #"role_departament" --> informacion redundante separada ya en las columnas "job_role" y "department"

df_hr.drop(["same_as_monthly_income","employee_count","role_departament"], axis = 1, inplace=True)

#### 1.3 Cambios de <u>**tipos de datos**</u>

- 1.3.1 Convertir de OBJETO a FLOAT _____________________________________________________________________________________________________________________________________________

In [61]:
# Revisando el contenido de las columnas que se han de convertir de objeto a float, se deben de quitar simbolos de dolar, espacios vacíos, cambiar los non available por numeros,
# sustituir las comas por puntos y entonces ya se podrá convertir a tipo float siempre teniendo en cuenta con np.nan los nulos
    # Como exploración se ha usado el método dtype y sample(10), en este caso no se incluyen ya que esta caja es sólo para convertir los datos
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# variable con las columnas a convertir
objt_float = ["daily_rate", "hourly_rate", "anual_income", "standard_hours","monthly_income"]                                     
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# función para quitar metacaracteres y espacios
def quitar_espacios_y_metacaracteres(df, columnas):                                                                                     
    for i in columnas:
        if i in df.columns:
            df[i] = df[i].astype(str).str.replace(r'[\!\.\^\$\*\+\?\{\}\[\]\(\)\|\\\s]', '', regex=True)
    return df

# Ejecutar función  
df_hr = quitar_espacios_y_metacaracteres(df_hr, objt_float)                                                                                                     
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# funcion para sustituir letras por nada
def sust_letras_por_nan(df,columnas):         
    for i in columnas:
        if i in df.columns:
            df[i] = df[i].astype(str).str.replace(r'[a-zA-Z]', '', regex=True)
        else:
            continue
    return df

df_hr = sust_letras_por_nan(df_hr, objt_float)   
# ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- 
# función para sustituir las comas por puntos ya que al pasar a float, se tienen que poder respetar los decimales en el sitio designado
def sust_comas_por_puntos (columnas):           
    return (columnas.replace(",", ".")) 

# Ejecutar función
df_hr[objt_float] = df_hr[objt_float].apply(sust_comas_por_puntos) 

# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Salto de caja
                                      

In [62]:
# funcion para cambiar de objeto a float
def cambiar_obj_a_float(df, columnas):
    for col in columnas:
        if col in df.columns:
            df[col] = df[col].astype(str).str.replace(',', '.', regex=False).str.strip()
            # Intentar convertir a float
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col] = df[col].fillna(df[col])
            df[col] = df[col].astype(float)
    return df

# Ejecutar la función
df_hr = cambiar_obj_a_float(df_hr, objt_float)

- 1.3.2 Convertir de OBJETO a INT ________________________________________________________________________________________________________________________________________________________

In [63]:
obj_int = ["age","employee_number","performance_rating","total_working_years","work_life_balance","years_in_current_role"]     

# Verificando caracteres especiales, metacaracteres y más

for columna in obj_int:
    metacaracteres= df_hr[columna].str.findall(r'[\!\,\.\^\$\*\+\?\{\}\[\]\(\)\|\\]')         # Buscar metacaracteres
    numeros= df_hr[columna].str.findall(r'\d+')                                               # Buscar números
    letras = df_hr[columna].str.findall(r'[a-zA-Z]')                                          # Buscar letras (mayúsculas y minúsculas)
    nulos = df_hr[columna].isnull().sum()                                                     # Buscar nulos
    vacio = df_hr[columna].str.findall(" ").count()                                           # Buscar espacios vacíos ya que parece que están causando conflicto

    # Sacar los caracteres de las variables
    metacaracteres_existentes = metacaracteres.explode().dropna().unique()
    numeros_existentes = numeros.explode().dropna().unique()
    letras_existentes = letras.explode().dropna().unique()


print(f"Columna: {columna.upper()}  Letras: {letras_existentes}    Números: {numeros_existentes}   Metacaracteres: {metacaracteres_existentes}   Cantidad de nulos: {nulos} Espacios vacíos:  {vacio}")
print("____" * 43)
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Ahora vamos a limpiar los datos
    # Age:                Hay que pasar de palabra a número y quitar espacios vacíos                          <-- Primero sustituimos palabras por núm y después lo incluimos en la def

    # employeenumber:     Hay que quitarle las comas y espacios vacíos, teniendo en cuenta los nulos (np.nan)        <-- def para comas por nada, fuera espacios, np.nan y cambio a int
    # PerformanceRating:  Hay que quitarle las comas y espacios vacíos, teniendo en cuenta los nulos (np.nan)        <-- def para comas por nada, fuera espacios, np.nan y cambio a int
    # TOTALWORKINGYEARS:  Hay que quitarle las comas y espacios vacíos, teniendo en cuenta los nulos (np.nan)        <-- def para comas por nada, fuera espacios, np.nan y cambio a int
    # WORKLIFEBALANCE:    Hay que quitarle las comas y espacios vacíos, teniendo en cuenta los nulos (np.nan)        <-- def para comas por nada, fuera espacios, np.nan y cambio a int
    # YearsInCurrentRole: Hay que quitarle las comas y espacios vacíos, teniendo en cuenta los nulos (np.nan)        <-- def para comas por nada, fuera espacios, np.nan y cambio a int
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------                                                         
# Creamos dict con las edades en letra por las edades en num para, con replace o apply sustituirlas
# Primero vemos cuales son los datos que tenemos que no sean números

print(df_hr["age"][df_hr['age'].str.contains(r'(?=.*[a-zA-Z])|(?=.*-)', na=False)])                            
print("____" * 43)                                                                                                 
# Sacamos los datos y creamos diccionario pasando de edad en letra a edad en número
cambio_age_letras = {"forty-seven":47,             
                     "fifty-eight":58,
                     "thirty-six":36,
                     "fifty-five":55,
                     "fifty-two":52,
                     "thirty-one":31,
                     "thirty":30,
                     "twenty-six":26,
                     "thirty-seven":37,
                     "thirty-two":32,
                     "twenty-four":24,
                     "thirty-two":32}
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Ahora sustituimos las edades en letra por las edades en número

def sustituir_edad(df, col, dict):                                                                                     
    return df[col].replace(dict)

# Aplicar la función
df_hr['age'] = sustituir_edad(df_hr, 'age', cambio_age_letras)                                                                                              
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# función para sustituir las comas por puntos ya que al pasar a float, se tienen que poder respetar los decimales en el sitio designado
def eliminar_comas (columnas):                  
    return columnas.str.replace(",", ".", regex=False)

# Ejecutar función
df_hr[obj_int] = df_hr[obj_int].apply(eliminar_comas)                                                                                                                  
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
def cambiar_obj_a_int(df, columnas):
    for i in columnas:
        if pd.api.types.is_object_dtype(df[i]):
            
            df[i] = pd.to_numeric(df[i], errors='coerce').astype(float)     # Convertir a float primero para manejar NaN
            
            df[i] = df[i].fillna(0).astype(int)   # Luego, convertir a int, llenando NaN con un valor adecuado (ejemplo: 0)
    return df

df_hr = cambiar_obj_a_int(df_hr, obj_int)   
df_hr[obj_int].dtypes                                                                                   

Columna: YEARS_IN_CURRENT_ROLE  Letras: []    Números: ['13' '0' '12' '11' '7' '6' '4' '3' '2' '1']   Metacaracteres: [',']   Cantidad de nulos: 1580 Espacios vacíos:  34
____________________________________________________________________________________________________________________________________________________________________________
31       forty-seven
79       fifty-eight
216       thirty-six
357       fifty-five
411        fifty-two
426       thirty-one
593           thirty
1198      twenty-six
1226    thirty-seven
1293      thirty-two
1325     twenty-four
1584      thirty-two
Name: age, dtype: object
____________________________________________________________________________________________________________________________________________________________________________


age                      int32
employee_number          int32
performance_rating       int32
total_working_years      int32
work_life_balance        int32
years_in_current_role    int32
dtype: object

- 1.3.3 Convertir de INT a FLOAT _________________________________________________________________________________________________________________________________________________________

In [64]:
# Creando variable 
int_a_float = ["distance_from_home", "monthly_rate", "percent_salary_hike", "stock_option_level"]                                                                           

# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Función para cambiar de tipo INT a FLOAT

def convertir_int_a_float(df, int_a_float):                                                                                                
    for columna in int_a_float:
        if columna in df.columns and pd.api.types.is_numeric_dtype(df[columna]):
            df[columna] = df[columna].astype(float)
            print(f"Columna {columna} convertida a float")

convertir_int_a_float(df_hr, int_a_float)


Columna distance_from_home convertida a float
Columna monthly_rate convertida a float
Columna percent_salary_hike convertida a float
Columna stock_option_level convertida a float


- 1.3.4 Convertir de FLOAT a INT _________________________________________________________________________________________________________________________________________________________

In [65]:
float_a_int = ["education", "job_level","number_children"]
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
def convertir_float_a_int(df, float_a_int):
    for columna in float_a_int:
        if columna in df.columns and pd.api.types.is_numeric_dtype(df[columna]):

            # Convertir a int, manteniendo NaN
            df[columna] = df[columna].astype('Int64')

convertir_float_a_int(df_hr, float_a_int)

### 2. Función de cambios de <span style="color:#00bf63">**registros**</span>

- 2.0 **.strip()** a todos los registros

In [66]:
# Hemos detectado que hay muchas columnas con espacios adelante y atrás, para cerciorarnos de que esto no sea un problema, se los quitaremos a todas las columnas
# El método strip() solo elimina los espacios en blanco al principio y al final de la cadena. No afecta a los espacios que hay entre las palabras.

# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Creando variable con los nombres de las columnas
variable_columnas_categoricas = ['attrition', 'business_travel', 'department', 'education_field', 'gender', 'job_role', 'marital_status', 'over18', 'over_time', 'remote_work']
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Función para quitar los espacios de delante y detrás de los registros

def strip_registros(df, columnas):
    for col in columnas:
        if col in df.columns:
            df[col] = df[col].str.strip()
    return df

# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Se detecta que gender está como int64 cuando debería de ser object
# El método strip() sólo funciona con tipo string, y transformándlo a "O", sa problemas, por lo que lo transformamos a string Python

df_hr["gender"] = df_hr["gender"].astype("string")

df_hr["gender"].dtype

# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Ejecutar función

df_hr = strip_registros(df_hr,variable_columnas_categoricas)
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Para unificar criterios, cambiamos el tipo de gender de string Python a object.
# Esto debía de ser hecho en este orden. string-cambio con método string-cambio a object

df_hr["gender"] = df_hr["gender"].astype("O")

- 2.1 Columna **gender**

In [67]:
# Función de cambio de registros --> columna ["Gender"]
# donde "object" == 0 --> "Male"
# donde "object" == 1 --> "Female"

def categorizar_gender(int):
    
    if int == "0":
        return "Male"
    
    elif int == "1":
        return "Female"
    
    else:
        pass

# Aplicamos el cambio directamente al df_hr usando el método `apply()`
df_hr["gender"] = df_hr["gender"].apply(categorizar_gender)

- 2.2 Columna **remote_work**

In [68]:
# Función de cambio de registros --> columna ["Remote_Work"]
# donde "valores" == 0 --> "No"
# donde "valores" == 1 --> "Yes"
# donde "valores" == False --> "No"
# donde "valores" == True --> "Yes"
# donde "valores" == Yes --> "Yes"

# escribimos "0" y "1" entre comillas porque son de tipo object, no int !!!!

# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Valores unicos de la columna remote_work = ['Yes' '1' 'False' '0' 'True']
def categorizar_remotework(valores):
    
    if valores == "Yes":
        return "Yes"
    
    elif valores == "True":
        return "Yes"
    
    elif valores == "False":
        return "No"
    
    elif valores == "1":
        return "Yes"
    
    elif valores == "0":
        return "No"
    
# Aplicamos el cambio directamente al df_hr usando el método `apply()`

df_hr["remote_work"] = df_hr["remote_work"].apply(categorizar_remotework)


- 2.3 Columna **job_role**

In [69]:
# Función de cambio de registros --> columna ["Job_Role"] capitalizando la primera letra de cada palabra
# ya que contiene inconsistencias en el uso de mayúsculas y minúsculas

def corregir_job_role(df):
  df['job_role'] = df['job_role'].str.strip().str.title() 
  
  return df

df_hr = corregir_job_role(df_hr)

- 2.4 Columna **marital_status**

In [70]:
# "Marital_Status", "Role_Department"] errores tipograficos
def corregir_marital_status(registros):

    if pd.isna(registros):
        return registros # Mantenemos el valor nulo dejándolo como está hasta que tomemos una decisión como equipo
    # otra opción sería que por cada valor nulo, leído inicialmente como nan, nos devuelva "Unknown"
    # si eligiéramos esta opción el cambio en el código sería muy simple:
    # cambiar return registros POR return "Unknown"

    elif registros == "Married" or registros == "Marreid":
        return "Married"
    
    elif registros == "Divorced":
        return "Divorced"
    
    elif registros == "Single":
        return "Single"
    
    elif registros == "divorced":
        return "Divorced"

# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------    

# Aplicamos el cambio directamente al df_hr usando el método `apply()`
df_hr["marital_status"] = df_hr['marital_status'].apply(corregir_marital_status)

- 2.5 Columna **business_travel**

In [71]:
def cambiar_business_travel(valor):
    if valor == "travel_rarely":
        return "Travel Rarely"
    
    elif valor == "non_travel":
        return "Non Travel"
    
    elif valor == "non-travel":
        return "Non Travel"
    
    elif valor == "travel_frequently":
        return "Travel Frequently"
    
    else:
        return valor
    
# Aplicamos la función a la columna business_travel
df_hr["business_travel"] = df_hr["business_travel"].apply(cambiar_business_travel)



### 3. Rellenando <span style="color:#00bf63">**datos faltantes (nulos)**</span>

* 3.1 Columna **employee_number**

In [72]:
# contiene duplicados. 431 Nulos y 103 duplicados
len =df_hr['employee_number'].__len__()
#print(f"Qty de filas en la columna employee_number: {len}")
counts = df_hr['employee_number'].value_counts()
# -----------------------------------------------------------------------------
# Filtrar solo los duplicados que no sean 0
duplicados = counts[counts > 1]
# Crear un nuevo DataFrame con los duplicados y sus conteos
df_duplicados = pd.DataFrame(duplicados).reset_index()
df_duplicados.columns = ['employee_number', 'count']
df_duplicados
# -----------------------------------------------------------------------------
# Porcentaje de nulos
valor_elemento_0 = df_duplicados['count'].iloc[0]
porcentaje_nulo = round((valor_elemento_0 / len) * 100, 2)
#print(f"Porcentaje de nulos: {porcentaje_nulo}%")
# -----------------------------------------------------------------------------
# Porcentaje de los duplicados
valor_duplicados = df_duplicados['count'].iloc[1::]
duplicados_total = valor_duplicados.sum()
porcentaje_duplicados = round((duplicados_total / len) * 100, 2)
#print(f"Porcentaje de duplicados {porcentaje_duplicados}%")

# -----------------------------------------------------------------------------
# Obtener los números de empleados que son duplicados
num_duplicados = df_duplicados['employee_number'].tolist()

# Se determina que hay numeros duplicados y los ceros son también duplicados, para revisarlo, primero sacamos los ceros
df_filtrado = df_hr[df_hr['employee_number'].isin(num_duplicados)& (df_hr['employee_number'] != 0)]
df_filtrado = df_filtrado.sort_values(by='employee_number', ascending=False)
pd.set_option('display.max_rows',None)
df_filtrado

# -----------------------------------------------------------------------------
# Concatenar todas las columnas
df_hr['concatenado'] = df_hr.astype(str).agg(' '.join, axis=1)

# Filtrar duplicados en employee_number
duplicados = df_hr[df_hr['employee_number'].duplicated(keep=False)]

# Contar cuántas veces se repite cada employee_number
conteo_duplicados = duplicados['employee_number'].value_counts()

# Crear un DataFrame para mostrar el resultado
resultados = []

# Iterar sobre los duplicados y recolectar información
for emp_num, count in conteo_duplicados.items():
    filas = duplicados[duplicados['employee_number'] == emp_num]
    indices = filas.index.tolist()
    concatenados = filas['concatenado'].tolist()
    
    resultados.append({
        'employee_number': emp_num,
        'count': count,
        'indices': indices,
        'concatenados': concatenados
    })

# -----------------------------------------------------------------------------

# Convertir a DataFrame
resultados_df = pd.DataFrame(resultados)

# Ordenar el DataFrame por employee_number en orden descendente
resultados_df = resultados_df.sort_values(by='employee_number', ascending=False)

# Mostrar el resultado
#print(f"\nResultados de duplicados:\n {resultados_df}")

# -----------------------------------------------------------------------------
# Nos ha dado los índices de los duplicados (duplicado = todos los registros de una fila concatenados y duplicados por columna)

try:
    df_indices = pd.DataFrame(resultados, columns=['indices'])
    df_indices = df_indices.iloc[1:]  # Excluir la fila 0
except ValueError as e:
    print(f"Error al crear el DataFrame: {e}")
    df_indices = pd.DataFrame(columns=['indices'])  # Crear un DataFrame vacío

#print(f"El df de los indices a partir del primer indice es: {df_indices}")
# -----------------------------------------------------------------------------
# Acceder a todos los índices 2
indices2_eliminar = []

for indices in df_indices['indices']:
    indices2_eliminar.append(indices[-1])  # Agregar el último elemento

#print("Lista de últimos índices:", indices2_eliminar)

# -----------------------------------------------------------------------------
# Ahora vamos a eliminar las filas con número de indice {indices2_eliminar} del df_hr
df_hr = df_hr.drop(index=indices2_eliminar) 

#print("\n__________________________________________\nDataFrame después de eliminar las filas:\n__________________________________________\n", df_hr)

# -----------------------------------------------------------------------------
# Verificamos que ya no tenemos duplicados. Ojo! Hay 431 duplicados de ceros
duplicados = counts[counts > 1]
duplicados # Vemos que ya sólo hay 431 
#df_hr

# -----------------------------------------------------------------------------
# Ahora tenemos que ver si hay registros duplicados entre los cero PERO en la siguiente caja


employee_number
0       431
482       2
530       2
507       2
517       2
522       2
523       2
525       2
526       2
544       2
501       2
725       2
724       2
722       2
721       2
717       2
710       2
502       2
495       2
663       2
440       2
734       2
733       2
728       2
423       2
424       2
433       2
447       2
1140      2
455       2
460       2
465       2
470       2
475       2
478       2
682       2
653       2
747       2
1069      2
389       2
388       2
381       2
376       2
1044      2
1053      2
1195      2
621       2
1185      2
1162      2
1161      2
1160      2
1157      2
1131      2
401       2
416       2
430       2
438       2
446       2
448       2
454       2
458       2
476       2
483       2
488       2
500       2
534       2
586       2
595       2
608       2
616       2
742       2
1135      2
762       2
1849      2
972       2
990       2
1927      2
1911      2
1898      2
1869      2
1821      2
966       2


* 3.1.2 Columna **employee_number**

In [73]:
# Ahora tenemos que ver si hay registros duplicados entre los cero
        # Eliminar la columna concatenado y cambiar los cero por null
# ------------------------------------------------------------------------------------------------------------
# Función de 0 a null
def cambiar_ceros_por_null(df, columna):
    df[columna] = df[columna].replace(0, np.nan)
    return df


df_hr = cambiar_ceros_por_null(df_hr, 'employee_number')
# ------------------------------------------------------------------------------------------------------------

# Eliminar columna concatenado
df_hr.drop(["concatenado"], axis = 1, inplace=True)

In [74]:
df_hr['employee_number'] = df_hr['employee_number'].astype(float)

In [75]:
def convertir_a_int(df, columna):
    # Reemplazar NaN por un valor (opcional, según tu lógica)
    # Puedes elegir mantener NaN o reemplazarlos por 0 u otro valor
    df[columna] = df[columna].fillna(0)  # Ejemplo: rellenar NaN con 0
    
    # Convertir a float primero
    df[columna] = df[columna].astype(float)
    
    # Luego convertir a int
    df[columna] = df[columna].astype(int)

    return df

df_hr = convertir_a_int(df_hr, 'employee_number')

- 3.2 La columna **distance_from_home**

In [76]:
# Asumimos que los valores negativos están escritos de manera incorrecta y el cambio que hacemos es convertirlos a negativo.
# valores negativos de distance_from_home a su valor absoluto:
df_hr["distance_from_home"] = df_hr["distance_from_home"].apply(lambda x: x if x >= 0 else -x)

- 3.4 La columna **hourly_rate** y **daily_rate**

In [77]:
# Hay poco más de un 5% de nulos, hemos comprobado con un boxplot que no hay valores outliers por lo que determinamos que podemos usar la media para rellenar los nulos.
media_hourly_rate = df_hr["hourly_rate"].mean()
df_hr["hourly_rate"] = df_hr["hourly_rate"].fillna(media_hourly_rate)
df_hr["hourly_rate"] = df_hr["hourly_rate"].round(2)
# ------------------------------------------------------------------------------------------------------------
# Hay poco más de un 7% de nulos, hemos comprobado con un boxplot que no hay valores outliers por lo que determinamos que podemos usar la media para rellenar los nulos.
media_daily_rate = df_hr["daily_rate"].mean()
df_hr["daily_rate"] = df_hr["daily_rate"].fillna(media_daily_rate)
df_hr["daily_rate"] = df_hr["daily_rate"].round(2)

* 3.5 INFIRIENDO VALORES **anual_income**

In [78]:
df_hr["anual_income"].isnull().sum()

799

ver outliers en grafica para identificarlos

donde anual_income = null
agrupar null de anual income con rol
sacar media de anual income por rol
asignar esa media a los nul del rola adecuado

In [79]:
# Buscamos un patrón del salario anual por puesto de trabajo. Ya que se infiere que los puestos de trabajo y el salario están relacionados por pertenecer a la misma categoría jerárquica.

salario_rol=pd.DataFrame(df_hr.groupby("job_role")["anual_income"].mean().round(2))
salario_rol
for rol in df_hr["job_role"]:
    if rol in salario_rol.index:
          df_hr.loc[(df_hr["job_role"] == rol) & df_hr["anual_income"].isnull(), "anual_income"] = salario_rol.loc[rol, "anual_income"]
df_hr.head(10)

,age,attrition,business_travel,daily_rate,department,distance_from_home,education,education_field,employee_number,environment_satisfaction,gender,hourly_rate,job_involvement,job_level,job_role,job_satisfaction,marital_status,anual_income,monthly_rate,num_companies_worked,over18,over_time,percent_salary_hike,performance_rating,relationship_satisfaction,standard_hours,stock_option_level,total_working_years,training_times_last_year,work_life_balance,years_at_company,years_in_current_role,years_since_last_promotion,years_with_curr_manager,year_birth,monthly_income,number_children,remote_work
0,51,No,NaN,684.0,NaN,6.0,3,NaN,162,1,Male,51.0,3,5,Research Director,3,NaN,19537.00,6462.0,7,Y,No,13.0,3,3,NaN,0.0,0,5,3,20,0,15,15,1972,1.000000e+09,<NA>,Yes
1,52,No,NaN,699.0,NaN,1.0,4,Life Sciences,259,3,Male,65.0,2,5,Manager,3,NaN,19999.00,5678.0,0,NaN,NaN,14.0,3,1,NaN,1.0,34,5,3,33,0,11,9,1971,1.000000e+09,<NA>,Yes
2,42,No,Travel Rarely,532.0,Research & Development,4.0,2,Technical Degree,319,3,Male,58.0,3,5,Manager,4,Married,19232.00,4933.0,1,NaN,No,11.0,3,4,NaN,0.0,22,3,0,22,0,11,15,1981,1.000000e+09,<NA>,Yes
3,47,No,Travel Rarely,359.0,NaN,2.0,4,Medical,0,1,Female,82.0,3,4,Research Director,3,Married,17169.00,26703.0,3,Y,NaN,19.0,3,2,NaN,2.0,0,2,0,20,0,5,6,1976,1.000000e+09,<NA>,No
4,46,No,NaN,1319.0,NaN,3.0,3,Technical Degree,0,1,Female,45.0,4,4,Sales Executive,1,Divorced,6982.03,7739.0,2,Y,No,12.0,3,4,NaN,1.0,0,5,3,19,0,2,8,1977,1.000000e+09,<NA>,No
5,48,No,NaN,117.0,Research & Development,22.0,3,Medical,1900,4,Female,58.0,3,4,Manager,4,NaN,17174.00,2437.0,3,NaN,No,11.0,3,2,NaN,1.0,0,3,3,22,0,4,7,1975,1.000000e+09,<NA>,Yes
6,59,No,NaN,1435.0,NaN,25.0,3,Life Sciences,81,1,Female,99.0,3,3,Sales Executive,1,NaN,6982.03,2354.0,7,Y,NaN,11.0,3,4,NaN,0.0,28,3,2,21,0,7,9,1964,1.000000e+09,<NA>,Yes
7,42,No,Travel Rarely,635.0,NaN,1.0,1,NaN,387,2,Male,99.0,3,2,Sales Executive,3,Married,6982.03,24532.0,1,NaN,No,25.0,4,3,NaN,0.0,20,3,3,20,0,11,6,1981,1.000000e+09,<NA>,No
8,41,No,NaN,1276.0,NaN,2.0,5,NaN,0,2,Female,91.0,3,4,Manager,1,Married,16595.00,5626.0,7,NaN,No,16.0,3,2,NaN,1.0,22,2,3,18,0,11,8,1982,1.000000e+09,<NA>,Yes
9,41,No,Travel Frequently,840.0,NaN,9.0,3,NaN,999,1,Male,64.0,3,5,Research Director,3,NaN,16089.84,3735.0,2,NaN,No,17.0,3,2,NaN,1.0,21,2,4,18,0,0,11,1982,1.000000e+09,<NA>,No


- 3.6 La columna **monthly_income**

In [80]:
# La columna salary, está en desuso ya que la recogida fue incorrecta. Aprovechamos la columna y la renombramos monthly_income y le sacamos los datos, 
# dividiendo el salario bruto anual que está en anual_income


def añadir_monthly_income(df):
   
   df["monthly_income"] = 0

   df["monthly_income"] = df["anual_income"] / 12

añadir_monthly_income(df_hr)

df_hr["monthly_income"] = df_hr["monthly_income"].round(2)

- 3.7 La columna **marital_status**,   **over_time**,  **department**, **education_field**, **business_travel**

In [81]:
# SI TIENE NULOS: decisión tomada en equipo: asignar "Unknown"

df_hr["marital_status"] = df_hr["marital_status"].fillna("Unknown")
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------    
# Los nulos se rellenan con "Unknown" puesto que no se puede inferir si han hecho o no hrs. extra los nulos
df_hr["over_time"] = df_hr["over_time"].fillna("Unknown")
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------    
# Se toma la decisión de mantener la columna por su alto impacto en la comparación con el resto de columnas al momento de sacar conclusiones sobre el trabajo de los empleados y su satisfacción
df_hr["department"] = df_hr["department"].fillna("Unknown")
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------    
#SI TIENE NULOS: signar "Unknown", ya que "Other" lo entiendo como "otro diferente a las opciones dadas" pero puede que el valor nulo sea una de esas opciones dadas y no "Other"
df_hr["education_field"] = df_hr["education_field"].fillna("Unknown")

# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------    
# Se sustituyen los nulos o nan por Unknown 
df_hr["business_travel"] = df_hr["business_travel"].fillna("Unknown")

- 3.8 La columna **over18**

In [82]:
def calcular_mayor_edad(fecha_nacimiento):
    edad = datetime.now().year - fecha_nacimiento
    if edad >= 18:
        return 'Yes'
    else:
        return 'No'

# Aplicamos la función a la columna 'year_birth' para rellenar 'over18'
# Usando la columna "year_birth", hemos rellenado los datos faltantes haciendo un cálculo
df_hr['over18'] = df_hr['year_birth'].apply(calcular_mayor_edad)

- 3.9 Columnas **education**, **job_involvement**, **job_level** y **job_satisfaction**: unificación de criterios de rangos

In [83]:
# Columna: **job_level**

#* 0 - Muy Insatisfecho
#* 1 - Insatisfecho
#* 2 - Neutral
#* 3 - Satisfecho
#* 4 - Muy Satisfecho
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------    
#Columna: **education**

#* 0- Educación secundaria obligatoria
#* 1-Título Técnico o Grado de formación profesional
#* 2- Título Universitario
#* 3- Título de Posgrado/Máster
#* 4- Doctorado
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------    
#Columnas: **job_involvement** y **job_satisfaction**

#* 0- Nulo: El empleado no tiene implicación en el trabajo
#* 1- Bajo: El empleado tiene un bajo nivel de implicación; su compromiso y esfuerzo son mínimos.
#* 2- Razonable: El empleado está comprometido, pero no totalmente involucrado.
#* 3- Alto: El empleado muestra un nivel alto de compromiso y dedicación en el trabajo.
#* 4- Muy Alto: El empleado está completamente involucrado y comprometido con el trabajo, dedicando todo su esfuerzo y atención.
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------    

def ajustar_rango(val, rango_permitido):
    if val in rango_permitido:
        return val
    else:
        return None  # decidimos qué hacer con los valores que no están en el rango (por ejemplo, dejarlos como None o NaN)

# Aplicación de la función para las columnas simples:
# Definimos los rangos permitidos para cada columna
rango_education = [0, 1, 2, 3, 4]
rango_job_involvement = [0, 1, 2, 3, 4]
rango_job_level = [0, 1, 2, 3, 4]
rango_job_satisfaction = [0, 1, 2, 3, 4]

# Aplicamos la función a cada columna
df_hr["education"] = df_hr["education"].apply(ajustar_rango, rango_permitido = rango_education).astype('Int64')
df_hr["job_involvement"] = df_hr["job_involvement"].apply(ajustar_rango, rango_permitido = rango_job_involvement).astype('Int64')
df_hr["job_level"] = df_hr["job_level"].apply(ajustar_rango, rango_permitido = rango_job_level).astype('Int64')
df_hr["job_satisfaction"] = df_hr["job_satisfaction"].apply(ajustar_rango, rango_permitido = rango_job_satisfaction).astype('Int64')

- 3.10 Columnas: **education** y **job_level**

In [84]:
#Gestión nulos en "education" y "job_level"

# Hay un porcentaje de menos de 3.5% nulos.
# Usaremos la media para gestionar los nulos ya que no contiene valores atípicos.
# Redondearemos la media ya que, aunque la distribución esté sesgada a la derecha, esto no afectará significativamente la interpretación de los resultados
# ya que los valores son discretos y el sesgo no es extremo.
media_education = np.round(df_hr["education"].mean())
df_hr["education"] = df_hr["education"].fillna(media_education).astype(int)

# ------------------------------------------------------------------------------------------------------------

# Hay un porcentaje de poco menos del 5% de nulos.
# Siendo la distribución de los datos relativamente concentrada, ligeramente sesgada y habiendo un posible outlier, la mediana es la opción más robusta.
mediana_job_level = df_hr["job_level"].median()
df_hr["job_level"] = df_hr["job_level"].fillna(mediana_job_level)

- 3.11 Columna:  **environment_satisfaction**: unificación de criterios

In [85]:
# Ajustamos environment_satisfaction a un rango válido
# esta columna tiene valores erróneos (valores entre 1 y 49), debemos quedarnos con la primera cifra para reescalarlos. PO ok.
def ajustar_environment_satisfaction(val):
    if val in range(1, 50):
        return int(str(val)[0])  # Nos quedamos con la primera cifra
    else:
        return None  # Para valores fuera de rango, se puede decidir cómo manejarlo (por ejemplo, asignar None)
df_hr["environment_satisfaction"] = df_hr["environment_satisfaction"].apply(ajustar_environment_satisfaction)


- 3.12 Columnas:  **performance_rating** y **work_life_balance**: unificación de criterios

In [86]:
# Ajustar performance_rating y work_life_balance
# Para estas columnas, vamos a ajustar los valores para que estén dentro del rango [0, 4], como ha indicado Carla.
# Función para ajustar performance_rating y work_life_balance:

rango_performance_rating = [0, 1, 2, 3, 4]
rango_work_life_balance = [0, 1, 2, 3, 4]

# Aplicamos la función de ajuste para ambas columnas
df_hr["performance_rating"] = df_hr["performance_rating"].apply(ajustar_rango, rango_permitido = rango_performance_rating)
      
df_hr["work_life_balance"] = df_hr["work_life_balance"].apply(ajustar_rango, rango_permitido = rango_work_life_balance)

In [87]:
df_hr.sample(5)

,age,attrition,business_travel,daily_rate,department,distance_from_home,education,education_field,employee_number,environment_satisfaction,gender,hourly_rate,job_involvement,job_level,job_role,job_satisfaction,marital_status,anual_income,monthly_rate,num_companies_worked,over18,over_time,percent_salary_hike,performance_rating,relationship_satisfaction,standard_hours,stock_option_level,total_working_years,training_times_last_year,work_life_balance,years_at_company,years_in_current_role,years_since_last_promotion,years_with_curr_manager,year_birth,monthly_income,number_children,remote_work
515,39,No,Unknown,1462.0,Sales,6.0,3,Unknown,1588,4,Male,38.0,4,3,Sales Executive,3,Married,8237.00,4658.0,2,Yes,No,11.0,3,1,NaN,1.0,11,3,3,7,0,7,6,1984,686.42,<NA>,Yes
288,38,No,Unknown,1084.0,Unknown,29.0,3,Unknown,0,4,Male,54.0,3,2,Manufacturing Director,4,Married,6261.00,4185.0,3,Yes,No,18.0,3,1,NaN,1.0,9,3,1,7,0,1,7,1985,521.75,<NA>,Yes
1078,23,No,Unknown,977.0,Unknown,10.0,3,Unknown,1592,4,Male,45.0,4,1,Research Scientist,3,Married,3314.25,12826.0,2,Yes,No,16.0,3,4,NaN,1.0,4,2,3,2,0,2,2,2000,276.19,<NA>,Yes
556,33,No,Non Travel,530.0,Unknown,16.0,3,Life Sciences,0,3,Female,36.0,3,2,Sales Executive,4,Unknown,5368.00,16130.0,1,Yes,Yes,25.0,4,3,80.0,1.0,7,2,3,6,0,1,2,1990,447.33,<NA>,No
776,42,No,Unknown,355.0,Unknown,10.0,4,Technical Degree,1854,3,Male,38.0,3,1,Research Scientist,3,Married,3314.25,6161.0,3,Yes,No,22.0,4,2,NaN,2.0,10,1,2,6,0,3,3,1981,276.19,<NA>,No


In [88]:
df_hr['employee_number'] = df_hr['employee_number'].astype(int)
df_hr.dtypes

age                             int32
attrition                      object
business_travel                object
daily_rate                    float64
department                     object
distance_from_home            float64
education                       int32
education_field                object
employee_number                 int32
environment_satisfaction        int64
gender                         object
hourly_rate                   float64
job_involvement                 Int64
job_level                       Int64
job_role                       object
job_satisfaction                Int64
marital_status                 object
anual_income                  float64
monthly_rate                  float64
num_companies_worked            int64
over18                         object
over_time                      object
percent_salary_hike           float64
performance_rating              int64
relationship_satisfaction       int64
standard_hours                float64
stock_option

### <span style="color:#00bf63">**Datos Apartados**</span> 

- Columnas **number_children, years_in_current_role**

    - **"number_children"** ➡️ 100% NULOS decidimos apartarla porque la consideramos importante para analizar la satisfacción del empleado (circunstancias externas / conciliacion laboral)
    - **"years_in_current_role"** ➡️ 97% NULOS decidimos apartarla porque la consideramos útil para estudiar la satisfacción del cliente en cuanto a la rotación

_____
## <span style="color:#00bf63">**Guardando el CSV**</span> cuando los datos estén completamente limpios y rellenados

In [89]:
df_hr.to_csv("HR_DATA_CLEAN.csv",index=False)